This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
GRAPH_NAME = "breakeven_jobs"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

GRAPH_NAME = "breakeven_inflation_10y"

GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [24]:
today = dt.date.today()

jobs_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "Book1.xlsx")

filt_jobs_df = jobs_df.rename(columns={
    "Author's estimates, smoothed": "Estimated Breakeven Jobs Added (Cheremukhin)",
    "Payroll gains": "Actual Jobs Added",
    "observation_date": 'Date'
    }
).set_index('Date')[["Estimated Breakeven Jobs Added (Cheremukhin)", 'Upper bound', 'Lower bound', 'Actual Jobs Added']].dropna()

filt_jobs_df = filt_jobs_df[filt_jobs_df.index >= '2025-01-01']


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [28]:
fig = go.Figure()

fig.add_trace(
        go.Scatter(
            x=filt_jobs_df.index,
            y=filt_jobs_df['Estimated Breakeven Jobs Added (Cheremukhin)'],
            mode='lines',
            name='Estimated Breakeven'
        )
    )

fig.add_trace(
        go.Scatter(
            x=filt_jobs_df.index,
            y=filt_jobs_df['Upper bound'],
            mode='lines',
            name='',
            line=dict(dash='dot', color='#7095cf'),
            showlegend=False  # hides this trace from the legend
        )
    )

fig.add_trace(
        go.Scatter(
            x=filt_jobs_df.index,
            y=filt_jobs_df['Lower bound'],
            mode='lines',
            name='',
            line=dict(dash='dot', color='#7095cf'),
            showlegend=False  # hides this trace from the legend
        )
    )

fig.add_trace(
        go.Bar(
            x=filt_jobs_df.index,
            y=filt_jobs_df['Actual Jobs Added'],
            name='Actual Jobs Added',
              marker=dict(color='#CF8B40')
        )
    )

fig.update_layout(
    title = dict(text = 'Breakeven Jobs Creation <br><sup>Monthly, Seasonally Adjusted</sup>'),
    xaxis_title="Date",
    yaxis_title="Change in Nonfarm Payrolls",
    legend=dict(orientation='h', y=-0.6, x=0.7),
    height = 350,
    width = 600
)

fig.update_yaxes(
    tickformat=".0f",
    ticksuffix="k"
)

fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
    title_text=None,
)

ut.add_end_labels(fig = fig)

fig.show()

fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")